In [1]:
from ont_fast5_api.fast5_interface import get_fast5_file
import pandas as pd
from Settings import *
from HelperFunctions import fullNormalise
import LSTMAutoencoder
import numpy as np
from numpy.linalg import norm
import time

1 - 00014eb4-4e2c-4087-ac24-57dea735b7b4

2 - 601f7bc7-3c09-4a78-a9b7-097bddcde809

3 - 1b6939ba-4a35-4696-bc6f-2ddb3368266e

4 - 95c03c50-57a5-4092-9a5f-87182d06a12c

5 - b6d8845b-3eec-42f6-9510-b8ba1fc1ec44


1 <--> 2    90.1

3 <--> 4    91.2

4 <--> 5    95.9 or 90.1


In [2]:
filepath = "../../similar_testdata/similar_squiggles.fast5"

def loadReads():

    print("reading squiggles file...")
    reads = []
    ids = []
    with get_fast5_file(filepath, mode="r") as f5:
        for read in f5.get_reads():
            if read.read_id in subset:
                raw_data = read.get_raw_data()
                reads.append(raw_data)
                ids.append(read.read_id)

    return reads, ids

In [3]:
squiggles, ids = loadReads()
ids

reading squiggles file...


['00014eb4-4e2c-4087-ac24-57dea735b7b4',
 '1b6939ba-4a35-4696-bc6f-2ddb3368266e',
 '601f7bc7-3c09-4a78-a9b7-097bddcde809',
 '95c03c50-57a5-4092-9a5f-87182d06a12c',
 'b6d8845b-3eec-42f6-9510-b8ba1fc1ec44']

In [4]:
model = LSTMAutoencoder.LSTM()
model.load_state_dict(torch.load(model_path))
model.eval()            #setting model to "predict mode"

LSTM(
  (lstm): LSTM(1, 200)
  (fc): Linear(in_features=200, out_features=1, bias=True)
)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
stateHs = []
stateCs = []
for i in range(len(ids)):
    print(f"embedding {i}...")
    s = time.time()
    x = fullNormalise(squiggles[i])

    x = torch.tensor(scaler.fit_transform(x.reshape(-1, 1))).squeeze(-1)
    x = x.type(torch.FloatTensor)


    state_h = torch.zeros(1, hidden_size)
    state_c = torch.zeros(1, hidden_size)

    with torch.no_grad():
        for value in x:
            pred, (state_h, state_c) = model(value, (state_h, state_c))

        
    
    stateHs.append(state_h.squeeze(0))
    stateCs.append(state_c.squeeze(0))
    print(f"Took {time.time()-s}s.")

    

embedding 0...
Took 14.736856937408447s.
embedding 1...
Took 24.241739511489868s.
embedding 2...
Took 35.803487062454224s.
embedding 3...
Took 26.976184844970703s.
embedding 4...
Took 14.376818418502808s.


In [6]:
print(stateCs[1].shape)
print(stateCs[1].squeeze(0))


torch.Size([200])
tensor([-1.0935e-02,  9.7726e-03, -2.1554e-03,  1.1428e-01, -7.9250e-03,
        -1.6848e-02,  2.9744e-02, -6.5192e-02,  1.8159e-01,  2.9580e-02,
         5.7526e-02,  3.0739e-02, -5.8726e-03, -6.2711e-02, -3.3663e-02,
         1.7028e-01,  6.5278e-02,  4.5522e-02, -3.7022e-03,  4.6695e-01,
         6.2273e-02,  5.2979e-02, -1.8669e-02, -1.0796e-01, -8.2114e-01,
         1.2053e-02, -3.6020e-02, -1.6950e-02, -2.9409e-01, -2.2709e-03,
        -5.4857e-02, -8.7643e-02,  8.3787e-02, -7.6077e-02,  3.8581e-02,
         9.8926e-02,  7.8070e-03,  1.5048e-01, -1.3851e-01, -2.3242e-02,
         7.3932e-02,  1.2487e-01, -1.9899e-01,  2.4910e-01,  7.2757e-03,
        -2.0467e-02, -6.7283e-02, -1.8049e-01, -2.7946e-03,  4.5130e-02,
         9.5316e-02, -6.8784e-04,  6.4018e-02,  1.0933e-01,  5.5900e-01,
        -2.1331e-02,  1.7065e-02, -5.0297e-03,  9.1622e-02, -1.4064e-01,
         3.9317e-02,  8.2144e-02,  1.6671e-02,  2.0089e-02, -3.0577e-03,
         1.1806e-02,  6.9573e-03,

In [7]:
similartiesH = {}
similartiesC = {}

for i in range(len(ids)):
    innerDictH = {}
    innerDictC = {}
    for s in range(len(ids)):
        if s != i:
            innerDictH[ids[s]] = str(norm(stateHs[i]-stateHs[s]))#str(np.dot(stateHs[i],stateHs[s])/(norm(stateHs[i])*norm(stateHs[s])))
            innerDictC[ids[s]] = str(norm(stateCs[i]-stateCs[s]))#str(np.dot(stateCs[i],stateCs[s])/(norm(stateCs[i])*norm(stateCs[s])))


    similartiesH[ids[i]] = innerDictH.copy()
    similartiesC[ids[i]] = innerDictC.copy()


In [8]:
import json

with open("hSimilarties.json", "w") as file:
    json.dump(similartiesH, file)

with open("cSimilarties.json", "w") as file:
    json.dump(similartiesC, file)